In [1]:
from utils.model_utils import load_model

In [2]:
model_config = {
    "model_path": "zhihan1996/DNA_bert_6"
}

_, tokenizer, _ = load_model(model_config, return_model=True)

Running the model on CUDA


Downloading:   0%|          | 0.00/359M [00:00<?, ?B/s]

Some weights of the model checkpoint at zhihan1996/DNA_bert_6 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNA_bert_6 a

zhihan1996/DNA_bert_6 loaded


In [ ]:
# process the data using the tokenizer
# use the SeqDataset class to create a dataset


In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
val_encodings = tokenizer(X_val, truncation=True, padding=True)

train_dataset = SeqDataset(train_encodings, y_train)
val_dataset = SeqDataset(val_encodings, y_val)

In [ ]:
results_dir = f'./results/classification/{model_name}/'
EPOCHS = 10
bs = 8

metric = evaluate.load('accuracy')

def compute_acc(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir=results_dir+'training_results',          # output directory
    num_train_epochs=EPOCHS,              # total number of training epochs
    per_device_train_batch_size=bs,  # batch size per device during training
    per_device_eval_batch_size=bs,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    # logging_dir=results_dir+'training_logs',            # directory for storing logs
    logging_steps=8,
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy = 'epoch'
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics = compute_acc,
    tokenizer=tokenizer
)




In [ ]:
trainer.train()
trainer.evaluate()
trainer.save_model(results_dir)